In [1]:
import numpy as np
import pandas as pd
import astropy.io.fits as fits
import healpy as hp
from scipy import integrate
import matplotlib.pyplot as plt
from astropy.cosmology import Planck15 as cosmo
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from completeness import create_completeness_dataframe

In [19]:
gkvInputCatv02_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/GAMA/gkvInputCatv02/gkvInputCatv02.fits')[1].data
SM_cat_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/GAMA/merged/StellarMass-gkvScience/mergedStellarMass-gkvScience')[1].data
efeds_raw = fits.open('/home/farnoosh/farnoosh/Master_Thesis_all/Data/eFEDS/cluster_catalogue_Liu/eFEDS_clusters_V3.2.fits')[1].data
vdisp, vdisp_err = np.genfromtxt('/home/farnoosh/Desktop/data/vdisp.ascii', skip_header=1, usecols=(8, 9), missing_values='--', filling_values=np.nan, unpack=True)

In [ ]:
""" efeds
    RA: 126.6108 , 145.0307
    DEC: -2.56405 , 5.70157
            """

In [3]:
galaxy_mask = (
         (SM_cat_raw['uberclass'] == 1 ) & #galaxy
         (SM_cat_raw['duplicate'] == False) &
         (SM_cat_raw['mask'] == False) &
         (SM_cat_raw['NQ'] > 2) &
         (SM_cat_raw['SC'] > 7) &
         (SM_cat_raw['mstar'] > 0) &
         (SM_cat_raw['RAcen'] > 126.5) &
         (SM_cat_raw['RAcen'] < 145.1) &
         (SM_cat_raw['Deccen'] > -2.6) &
         (SM_cat_raw['Deccen'] < 5.8) &
         (SM_cat_raw['Z'] < 1.31) &
         (SM_cat_raw['starmask'] == False)
)
galaxy_catalog = SM_cat_raw[galaxy_mask]
print('number of the galaxies: ',len(galaxy_catalog))

number of the galaxies:  36801


In [4]:
H0 = 70
c = 3e5
arcmin_to_rad = np.pi / 180 / 60

In [5]:
def distance_from_redshift(z):
    return c * z / H0

def cluster_volume(cluster_radius_Mpc):
    return (4/3) * np.pi * cluster_radius_Mpc**3

# calculate distance between two points given their coordinates
def angular_distance(RA1, DEC1, RA2, DEC2):
    ra_diff = np.radians(RA2 - RA1)
    dec_diff = np.radians(DEC2 - DEC1)
    a = np.sin(dec_diff / 2)**2 + np.cos(np.radians(DEC1)) * np.cos(np.radians(DEC2)) * np.sin(ra_diff / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return c

def compute_velocity_dispersion(recession_velocity):
    mean_velocity = np.mean(recession_velocity)
    velocity_differences = recession_velocity - mean_velocity
    sum_of_squared_differences = np.sum(velocity_differences**2)
    variance = sum_of_squared_differences / (len(recession_velocity) - 1)
    velocity_dispersion = np.sqrt(variance)
    return velocity_dispersion

def galaxies_within_clusters(galaxy_df, cluster_df):
    in_cluster_radius = np.zeros(len(galaxy_df), dtype=bool)
    galaxy_cluster_id = np.zeros(len(galaxy_df), dtype=int)
    galaxy_velocities = []

    for i, galaxy_row in galaxy_df.iterrows():
        galaxy_ra = galaxy_row['RA']
        galaxy_dec = galaxy_row['DEC']
        galaxy_z = galaxy_row['z']
        min_angular_sep = np.inf
        closest_cluster_id = -1
        recession_velocity = []

        for j, cluster_row in cluster_df.iterrows():
            cluster_ra = cluster_row['RA']
            cluster_dec = cluster_row['DEC']
            cluster_radius_rad = cluster_row['cluster_radius']
            cluster_z_border_min = cluster_row['z_border_min']
            cluster_z_border_max = cluster_row['z_border_max']

            if galaxy_z < cluster_z_border_min or galaxy_z > cluster_z_border_max:
                continue

            cos_angular_sep = np.sin(np.radians(galaxy_dec)) * np.sin(np.radians(cluster_dec)) \
                            + np.cos(np.radians(galaxy_dec)) * np.cos(np.radians(cluster_dec)) \
                            * np.cos(np.radians(galaxy_ra - cluster_ra))
            angular_sep_rad = np.arccos(cos_angular_sep)

            physical_sep_Mpc = angular_sep_rad * cluster_row['distance_Mpc']

            # Calculate the recession velocity of the galaxy from the cluster
            recession_velocity.append(H0 * physical_sep_Mpc)

            # Check if galaxy is within cluster radius and closest to the cluster
            if angular_sep_rad <= cluster_radius_rad and angular_sep_rad < min_angular_sep:
                min_angular_sep = angular_sep_rad
                closest_cluster_id = cluster_row['c_ID']

        # Update the flags and cluster IDs
        if closest_cluster_id != -1:
            in_cluster_radius[i] = True
            galaxy_cluster_id[i] = closest_cluster_id
            galaxy_velocities.extend(recession_velocity)

    velocity_dispersion = compute_velocity_dispersion(np.array(galaxy_velocities))
    return in_cluster_radius, galaxy_cluster_id, velocity_dispersion


In [21]:
cluster_df = pd.DataFrame()
cluster_df['z'] = efeds_raw['z'].byteswap().newbyteorder()
cluster_df['RA'] = (efeds_raw['RA']).byteswap().newbyteorder()    # degrees
cluster_df['DEC'] = (efeds_raw['DEC']).byteswap().newbyteorder()   # degrees
cluster_df['c_ID'] = efeds_raw['ID_SRC'].byteswap().newbyteorder()
cluster_df['distance_Mpc'] = distance_from_redshift(cluster_df['z'])
cluster_df['cluster_radius'] = (efeds_raw['R_SNR_MAX'] * arcmin_to_rad).byteswap().newbyteorder() # radian
cluster_df['cluster_radius_Mpc'] = cluster_df['distance_Mpc'] * np.tan(cluster_df['cluster_radius'])
cluster_df['cluster_volume_Mpc3'] = cluster_volume(cluster_df['cluster_radius_Mpc'])
vdisp_df = pd.DataFrame({'VDISP': vdisp})
vdisp_err_df = pd.DataFrame({'VDISP_ERR': vdisp_err})
cluster_df = pd.concat([cluster_df, vdisp_df, vdisp_err_df], axis=1)

In [25]:
# Count the number of NaN values in the 'VDISP' column
nan_count = cluster_df['VDISP'].isna().sum()

# Print the count of NaN values
print("Number of NaN values in 'VDISP' column:", nan_count)
print(100*nan_count/542)

Number of NaN values in 'VDISP' column: 260
47.97047970479705


In [7]:
galaxy_df = pd.DataFrame()
galaxy_df['z'] = galaxy_catalog['Z'].byteswap().newbyteorder()
galaxy_df['RA'] = galaxy_catalog['RAcen'].byteswap().newbyteorder()  #degrees
galaxy_df['DEC'] = galaxy_catalog['Deccen'].byteswap().newbyteorder()    #degrees
galaxy_df['g_ID'] = galaxy_catalog['uberID'].byteswap().newbyteorder()

In [20]:
# Calculate the redshift at the border
z_border_min = cluster_df['z'] - (cluster_df['cluster_radius_Mpc'] / cluster_df['distance_Mpc']) + 0.1 * (cluster_df['cluster_radius_Mpc'] / cluster_df['distance_Mpc'])
z_border_max = cluster_df['z'] + (cluster_df['cluster_radius_Mpc'] / cluster_df['distance_Mpc']) + 0.1 * (cluster_df['cluster_radius_Mpc'] / cluster_df['distance_Mpc'])
cluster_df['z_border_min'] = z_border_min
cluster_df['z_border_max'] = z_border_max

In [9]:
in_cluster_radius, galaxy_cluster_id, velocity_dispersion = galaxies_within_clusters(galaxy_df, cluster_df)

In [10]:
# Add the 'in_cluster_radius' and 'cluster_id' columns to the galaxy dataframe
galaxy_df['in_cluster_radius'] = in_cluster_radius
galaxy_df['cluster_id'] = galaxy_cluster_id

In [11]:
# Group galaxies by cluster ID and aggregate their IDs into a list
galaxies_per_cluster_with_ids = galaxy_df[galaxy_df['in_cluster_radius']].groupby('cluster_id')['g_ID'].agg(list)

In [12]:
# Print the number of galaxies associated with each cluster along with their IDs
for cluster_id, galaxy_ids in galaxies_per_cluster_with_ids.items():
    num_galaxies = len(galaxy_ids)
    galaxy_ids_str = ', '.join([str(galaxy_id) for galaxy_id in galaxy_ids])
    print(f"Cluster ID {cluster_id}: Number of galaxies = {num_galaxies} with IDs of: {galaxy_ids_str}")


Cluster ID 150: Number of galaxies = 1 with IDs of: 140010609901292
Cluster ID 152: Number of galaxies = 1 with IDs of: 135020664511413
Cluster ID 197: Number of galaxies = 1 with IDs of: 129020809511497
Cluster ID 298: Number of galaxies = 1 with IDs of: 140980622605907
Cluster ID 328: Number of galaxies = 8 with IDs of: 134000965407655, 134000981307856, 134000988507709, 134000995908028, 134000997207858, 134001014807824, 134000986307738, 134001037007521
Cluster ID 356: Number of galaxies = 2 with IDs of: 129980747104443, 129980747104360
Cluster ID 489: Number of galaxies = 4 with IDs of: 140020824108892, 140020828308716, 140020838909058, 140020851208733
Cluster ID 510: Number of galaxies = 1 with IDs of: 138001001609916
Cluster ID 534: Number of galaxies = 3 with IDs of: 138990590607637, 138990592407066, 138990553306987
Cluster ID 736: Number of galaxies = 1 with IDs of: 129020244401050
Cluster ID 810: Number of galaxies = 1 with IDs of: 140010179511440
Cluster ID 857: Number of galax

In [17]:
vdisp, vdisp_err = np.genfromtxt('/home/farnoosh/Desktop/data/vdisp.ascii', skip_header=1, usecols=(8, 9), missing_values='--', filling_values=np.nan, unpack=True)

vdisp_df = pd.DataFrame({'VDISP': vdisp})
vdisp_err_df = pd.DataFrame({'VDISP_ERR': vdisp_err})
cluster_df = pd.concat([cluster_df, vdisp_df, vdisp_err_df], axis=1)

            z          RA       DEC   c_ID  distance_Mpc  cluster_radius  \
0    0.161110  126.610799 -0.574787  28993    690.471429        0.000233   
1    0.257160  126.965471 -0.481638  11248   1102.114286        0.000215   
2    0.076155  127.036645 -0.167715   4800    326.378571        0.000765   
3    0.844900  127.085556 -0.122752   4169   3621.000000        0.000427   
4    0.319705  127.169202 -0.083552   7991   1370.164286        0.000389   
..        ...         ...       ...    ...           ...             ...   
537  0.024000  144.434057  2.760046   3372    102.857144        0.000362   
538  0.262250  144.627172  4.256396  10322   1123.928571        0.000751   
539  0.368715  144.909596  4.371716   1594   1580.207143        0.000482   
540  0.469977  145.024593  3.224726   4689   2014.187106        0.000273   
541  0.666382  145.030728  3.965204  11337   2855.922857        0.000336   

     cluster_radius_Mpc  cluster_volume_Mpc3  z_border_min  z_border_max  \
0          